In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

In [2]:
def generate_instance():
    # iç veri
    ay = np.max([1.0, np.round(np.random.normal(3, 1))]) if np.random.rand() > 0.93 else np.max([1.0, np.round(np.random.normal(28, 7))])
    kredi_karti_aldi = 1.0 if np.random.rand() > 0.99 else 0.0
    kredi_karti_sahibi = 1.0 if np.random.rand() > 0.7 else 0.0
    kredi_karti_ilk_ay_harcama = 0.0 if (kredi_karti_sahibi == 0.0 and kredi_karti_aldi == 0.0) else np.max([0.0, np.random.normal(1000.0, 250.0)])
    kredi_karti_harcama = 0.0 if (kredi_karti_sahibi == 0.0 and kredi_karti_aldi == 0.0) else np.max([0.0, np.random.normal(3000, 2000)])
    onceki_ay_kredi_karti_harcama = 0.0 if kredi_karti_sahibi == 0.0 else np.max([0.0, np.random.normal(3000, 2000)])
    toplam_varlik = 0.0 if np.random.rand() > 0.95 else np.max([0.0, np.random.normal(10000, 8000)])
    toplam_varlik_gecen_ay = 0.0 if ay < 2 or np.random.rand() > 0.5 else np.max([0.0, np.random.normal(10000, 8000)])
    toplam_varlik_onceki_3_ay_ortalama = 0.0 if ay < 4 else np.max([toplam_varlik_gecen_ay, np.random.normal(toplam_varlik_gecen_ay, 5000)])
    acik_kredi = 1.0 if np.random.rand() > 0.8 else 0.0
    onceki_ay_urun_sayisi = 0.0 if np.random.rand() > 0.995 else np.max([0.0, np.round(np.random.normal(4.0, 1.5))])
    urun_sayisi = 0.0 if np.random.rand() > 0.98 else np.max([0.0, np.round(np.random.normal(4.0, 1.5))])
    
    # performans ölçütleri
    yeni_musteri = 1.0 if ay <= 3.0 else 0.0
    capraz_satis = 1.0 if ay > 3.0 and (kredi_karti_sahibi == 1.0 and kredi_karti_ilk_ay_harcama > 100.0) and np.random.rand() > 0.8 else 0.0
    yukari_satis = 1.0 if ay > 3.0 and (toplam_varlik_gecen_ay > 10000 and toplam_varlik / toplam_varlik_onceki_3_ay_ortalama > 1.5) else 0.0
    
    devam_eden_musteri = 0.0
    terk = 0.0
    geri_kazanilmis_musteri = 0.0
    pasif_musteri = 0.0
    aktiflestirilmis_musteri = 0.0
    
    if ay > 3 and kredi_karti_sahibi == 0 and toplam_varlik == 0 and acik_kredi == 0:
        pasif_musteri = 1.0
    elif ay > 3 and urun_sayisi == 0:
        terk = 1.0
    elif ay > 3 and onceki_ay_urun_sayisi == 0.0 and urun_sayisi > 0:
        geri_kazanilmis_musteri = 1.0
    elif ay > 3 and toplam_varlik_gecen_ay == 0.0 and onceki_ay_kredi_karti_harcama == 0.0 and onceki_ay_urun_sayisi > 0 and urun_sayisi > 0 and toplam_varlik != 0 and toplam_varlik_onceki_3_ay_ortalama != 0 and np.random.rand() > 0.9:
        aktiflestirilmis_musteri = 1.0
    elif ay > 3:
        devam_eden_musteri = 1.0
        
    tavsiye = np.random.normal(6.5, 0.01) # bias
    tavsiye += yeni_musteri * np.random.normal(2, 0.1)
    tavsiye += capraz_satis * np.random.normal(1, 0.1)
    tavsiye += yukari_satis * np.random.normal(3, 0.001)
    tavsiye += devam_eden_musteri * np.random.normal(0.5, 0.5)
    tavsiye += terk * np.random.normal(-3.0, 0.001)
    tavsiye += geri_kazanilmis_musteri * np.random.normal(1.0, 0.5)
    tavsiye += aktiflestirilmis_musteri * np.random.normal(0.5, 0.5)
    tavsiye += pasif_musteri * np.random.normal(-1.5, 0.1)
    tavsiye = np.round(tavsiye)
    tavsiye = np.max([0.0, tavsiye])
    tavsiye = np.min([10.0, tavsiye])
    
    instance = [ay, kredi_karti_aldi, kredi_karti_sahibi, kredi_karti_ilk_ay_harcama, kredi_karti_harcama, onceki_ay_kredi_karti_harcama, toplam_varlik, toplam_varlik_gecen_ay, toplam_varlik_onceki_3_ay_ortalama,
    acik_kredi, onceki_ay_urun_sayisi, urun_sayisi, yeni_musteri, capraz_satis, yukari_satis, devam_eden_musteri, terk, geri_kazanilmis_musteri, pasif_musteri, aktiflestirilmis_musteri,
    tavsiye]
    
    return np.array(instance)
    
    
    
    

In [3]:
samples = np.array([generate_instance() for x in range(100000)])
columns = ["ay", "kredi_karti_aldi", "kredi_karti_sahibi", "kredi_karti_ilk_ay_harcama", "kredi_karti_harcama", "onceki_ay_kredi_karti_harcama", "toplam_varlik", "toplam_varlik_gecen_ay", "toplam_varlik_onceki_3_ay_ortalama",
    "acik_kredi", "onceki_ay_urun_sayisi", "urun_sayisi", "yeni_musteri", "capraz_satis", "yukari_satis", "devam_eden_musteri", "terk", "geri_kazanilmis_musteri", "pasif_musteri", "aktiflestirilmis_musteri",
    "tavsiye"]
data = pd.DataFrame(samples, columns = columns)

In [4]:
data["yeni_musteri"].sum() / data.shape[0]

0.04889

In [5]:
data["capraz_satis"].sum() / data.shape[0]

0.05664

In [6]:
data["yukari_satis"].sum() / data.shape[0]

0.01281

In [7]:
data["devam_eden_musteri"].sum() / data.shape[0]

0.81819

In [8]:
data["terk"].sum() / data.shape[0]

0.02537

In [9]:
data["geri_kazanilmis_musteri"].sum() / data.shape[0]

0.01268

In [10]:
data["pasif_musteri"].sum() / data.shape[0]

0.07973

In [11]:
data["aktiflestirilmis_musteri"].sum() / data.shape[0]

0.01514

In [12]:
data["tavsiye"].mean()

6.92485

In [13]:
data.to_csv("../data/generated.csv", sep="\t")